# Data Ingestion Pipeline

This notebook documents the process of loading raw telecom datasets
(15 CSV files and 2 GeoJSON files, ~1.5GB) into Google BigQuery.

## Steps
1. Upload raw files to Google Cloud Storage
2. Read files from GCS using Python
3. Load CSV and GeoJSON data into BigQuery tables

## Notes
- Raw data files are not included in this repository due to size constraints
- This notebook is for documentation and reproducibility purposes

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery

# --- AYARLAR ---
project_id = 'ny-project-480109' # Buranın doğruluğundan emin olun
dataset_name = 'telecom_italy_data' # BigQuery'de elle oluşturduğunuz dataset adı
bucket_name = 'telecom_italy'

# Client'ı proje ID ile başlatıyoruz
client = bigquery.Client(project=project_id)

csv_files = [
    "ISTAT_census_variables_2011.csv", "mi-to-provinces-2013-11-01.csv",
    "mi-to-provinces-2013-11-02.csv", "mi-to-provinces-2013-11-03.csv",
    "mi-to-provinces-2013-11-04.csv", "mi-to-provinces-2013-11-05.csv",
    "mi-to-provinces-2013-11-06.csv", "mi-to-provinces-2013-11-07.csv",
    "sms-call-internet-mi-2013-11-01.csv", "sms-call-internet-mi-2013-11-02.csv",
    "sms-call-internet-mi-2013-11-03.csv", "sms-call-internet-mi-2013-11-04.csv",
    "sms-call-internet-mi-2013-11-05.csv", "sms-call-internet-mi-2013-11-06.csv",
    "sms-call-internet-mi-2013-11-07.csv"
]

for file_name in csv_files:
    # İsimdeki geçersiz karakterleri temizle
    clean_table_name = file_name.replace('.csv', '').replace('-', '_').replace('.', '_')

    # TAM YOL: project_id.dataset_name.table_name
    table_id = f"{project_id}.{dataset_name}.{clean_table_name}"

    uri = f"gs://{bucket_name}/{file_name}"

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        autodetect=True,
        write_disposition="WRITE_TRUNCATE" # Varsa üstüne yazar
    )

    print(f"İşlem başlıyor: {file_name} -> {table_id}")

    try:
        load_job = client.load_table_from_uri(uri, table_id, job_config=job_config)
        load_job.result()  # Yükleme bitene kadar bekle
        print(f"✅ BAŞARILI: {clean_table_name}")
    except Exception as e:
        print(f"❌ HATA: {file_name} yüklenemedi. Detay: {e}")

İşlem başlıyor: ISTAT_census_variables_2011.csv -> ny-project-480109.telecom_italy_data.ISTAT_census_variables_2011
✅ BAŞARILI: ISTAT_census_variables_2011
İşlem başlıyor: mi-to-provinces-2013-11-01.csv -> ny-project-480109.telecom_italy_data.mi_to_provinces_2013_11_01
✅ BAŞARILI: mi_to_provinces_2013_11_01
İşlem başlıyor: mi-to-provinces-2013-11-02.csv -> ny-project-480109.telecom_italy_data.mi_to_provinces_2013_11_02
✅ BAŞARILI: mi_to_provinces_2013_11_02
İşlem başlıyor: mi-to-provinces-2013-11-03.csv -> ny-project-480109.telecom_italy_data.mi_to_provinces_2013_11_03
✅ BAŞARILI: mi_to_provinces_2013_11_03
İşlem başlıyor: mi-to-provinces-2013-11-04.csv -> ny-project-480109.telecom_italy_data.mi_to_provinces_2013_11_04
✅ BAŞARILI: mi_to_provinces_2013_11_04
İşlem başlıyor: mi-to-provinces-2013-11-05.csv -> ny-project-480109.telecom_italy_data.mi_to_provinces_2013_11_05
✅ BAŞARILI: mi_to_provinces_2013_11_05
İşlem başlıyor: mi-to-provinces-2013-11-06.csv -> ny-project-480109.telecom_ita

In [ ]:
pip install google-cloud-bigquery

In [ ]:
from google.cloud import bigquery
import json

def upload_geojson_to_bigquery(geojson_file_path, project_id, dataset_id, table_id):
    """
    GeoJSON dosyasını BigQuery'ye yükler

    Args:
        geojson_file_path: GeoJSON dosyasının yolu
        project_id: Google Cloud project ID
        dataset_id: BigQuery dataset ID
        table_id: BigQuery table ID
    """
    # BigQuery client oluştur
    client = bigquery.Client(project=project_id)

    # GeoJSON dosyasını oku
    print(f"Dosya okunuyor: {geojson_file_path}")
    with open(geojson_file_path, 'r', encoding='utf-8') as f:
        geojson_data = json.load(f)

    print(f"Toplam {len(geojson_data['features'])} feature bulundu")

    # GeoJSON features'ları satırlara dönüştür
    rows_to_insert = []

    # İlk feature'dan properties şemasını çıkar
    sample_properties = geojson_data['features'][0].get('properties', {}) if geojson_data['features'] else {}

    for idx, feature in enumerate(geojson_data['features']):
        row = {
            'feature_id': idx,
            'geometry_type': feature['geometry'].get('type', ''),
            'geometry_json': json.dumps(feature['geometry']),
            'properties_json': json.dumps(feature.get('properties', {}))
        }

        # Properties içindeki alanları da ekle
        properties = feature.get('properties', {})
        for key, value in properties.items():
            # BigQuery kolon adları için geçerli hale getir
            safe_key = key.replace(' ', '_').replace('-', '_').lower()
            row[f'prop_{safe_key}'] = str(value) if value is not None else None

        rows_to_insert.append(row)

    # Tablo referansı
    table_ref = f"{project_id}.{dataset_id}.{table_id}"

    # Tablo şeması - dinamik olarak properties'den oluştur
    schema = [
        bigquery.SchemaField("feature_id", "INTEGER"),
        bigquery.SchemaField("geometry_type", "STRING"),
        bigquery.SchemaField("geometry_json", "STRING"),
        bigquery.SchemaField("properties_json", "STRING"),
    ]

    # Properties'deki her alan için kolon ekle
    for key in sample_properties.keys():
        safe_key = key.replace(' ', '_').replace('-', '_').lower()
        schema.append(bigquery.SchemaField(f"prop_{safe_key}", "STRING"))

    # Tablo oluştur (yoksa)
    table = bigquery.Table(table_ref, schema=schema)
    try:
        table = client.create_table(table)
        print(f"✓ Tablo oluşturuldu: {table_ref}")
    except Exception as e:
        if "Already Exists" in str(e):
            print(f"✓ Tablo zaten mevcut: {table_ref}")
        else:
            print(f"! Uyarı: {e}")

    # Verileri yükle
    print(f"Veriler yükleniyor...")
    errors = client.insert_rows_json(table_ref, rows_to_insert)

    if errors:
        print(f"✗ Hatalar oluştu: {errors}")
        return False
    else:
        print(f"✓ {len(rows_to_insert)} satır başarıyla yüklendi: {table_ref}")
        return True

def main():
    import os
    import glob

    # Parametreler
    PROJECT_ID = "ny-project-480109"
    DATASET_ID = "telecom_italy_data"

    # Dataset'in var olduğundan emin ol
    client = bigquery.Client(project=PROJECT_ID)
    dataset_ref = f"{PROJECT_ID}.{DATASET_ID}"

    try:
        client.get_dataset(dataset_ref)
        print(f"✓ Dataset mevcut: {DATASET_ID}")
    except Exception as e:
        print(f"! Dataset oluşturuluyor: {DATASET_ID}")
        dataset = bigquery.Dataset(dataset_ref)
        dataset.location = "US"  # İsterseniz değiştirebilirsiniz
        client.create_dataset(dataset)
        print(f"✓ Dataset oluşturuldu: {DATASET_ID}")

    # Dosyaları bul
    possible_paths = [
        "/mnt/user-data/uploads/",
        "/home/claude/",
        "./",
        ""
    ]

    milano_file = None
    provinces_file = None

    for path in possible_paths:
        # Milano grid dosyasını ara
        if not milano_file:
            test_path = os.path.join(path, "milano-grid.geojson")
            if os.path.exists(test_path):
                milano_file = test_path

        # Italian provinces dosyasını ara
        if not provinces_file:
            test_path = os.path.join(path, "Italian_provinces.geojson")
            if os.path.exists(test_path):
                provinces_file = test_path

    # Tüm geojson dosyalarını listele
    all_geojson = []
    for path in possible_paths:
        all_geojson.extend(glob.glob(os.path.join(path, "*.geojson")))

    if all_geojson:
        print(f"\nBulunan GeoJSON dosyaları:")
        for f in all_geojson:
            print(f"  - {f}")
        print()

    if not milano_file:
        print("✗ milano-grid.geojson bulunamadı!")
        print("Lütfen dosyayı yükleyin veya doğru yolu belirtin.")
        return

    if not provinces_file:
        print("✗ Italian_provinces.geojson bulunamadı!")
        print("Lütfen dosyayı yükleyin veya doğru yolu belirtin.")
        return

    print("\n" + "="*60)
    print("GeoJSON dosyaları BigQuery'ye yükleniyor...")
    print("="*60 + "\n")

    # İlk GeoJSON dosyası - Milano Grid
    print("1. milano-grid.geojson yükleniyor...")
    print("-" * 60)
    success1 = upload_geojson_to_bigquery(
        geojson_file_path=milano_file,
        project_id=PROJECT_ID,
        dataset_id=DATASET_ID,
        table_id="milano_grid"
    )

    print("\n")

    # İkinci GeoJSON dosyası - Italian Provinces
    print("2. Italian_provinces.geojson yükleniyor...")
    print("-" * 60)
    success2 = upload_geojson_to_bigquery(
        geojson_file_path=provinces_file,
        project_id=PROJECT_ID,
        dataset_id=DATASET_ID,
        table_id="italian_provinces"
    )

    print("\n" + "="*60)
    print("ÖZET")
    print("="*60)
    print(f"Milano Grid: {'✓ Başarılı' if success1 else '✗ Başarısız'}")
    print(f"Italian Provinces: {'✓ Başarılı' if success2 else '✗ Başarısız'}")
    print("\nOluşturulan tablolar:")
    print(f"  - {PROJECT_ID}.{DATASET_ID}.milano_grid")
    print(f"  - {PROJECT_ID}.{DATASET_ID}.italian_provinces")
    print("="*60)

if __name__ == "__main__":
    main()

✓ Dataset mevcut: telecom_italy_data

Bulunan GeoJSON dosyaları:
  - ./milano-grid.geojson
  - ./Italian_provinces.geojson
  - milano-grid.geojson
  - Italian_provinces.geojson


GeoJSON dosyaları BigQuery'ye yükleniyor...

1. milano-grid.geojson yükleniyor...
------------------------------------------------------------
Dosya okunuyor: ./milano-grid.geojson
Toplam 10000 feature bulundu
✓ Tablo oluşturuldu: ny-project-480109.telecom_italy_data.milano_grid
Veriler yükleniyor...
✓ 10000 satır başarıyla yüklendi: ny-project-480109.telecom_italy_data.milano_grid


2. Italian_provinces.geojson yükleniyor...
------------------------------------------------------------
Dosya okunuyor: ./Italian_provinces.geojson
Toplam 110 feature bulundu
✓ Tablo oluşturuldu: ny-project-480109.telecom_italy_data.italian_provinces
Veriler yükleniyor...
✓ 110 satır başarıyla yüklendi: ny-project-480109.telecom_italy_data.italian_provinces

ÖZET
Milano Grid: ✓ Başarılı
Italian Provinces: ✓ Başarılı

Oluşturulan t